# Using xarray to read EBAS data

See more at http://ebas.nilu.no/
and https://ebas.nilu.no/thredds/

requirements
pip install threddsclient

The EBAS database collects observational data on atmospheric chemical composition and physical properties from a variety of national and international research projects and monitoring programs, such as ACTRIS, AMAP, EMEP, GAW and HELCOM, as well as for the Norwegian monitoring programs funded by the Norwegian Environment Agency, the Ministry of Climate and Environment and NILU – Norwegian Institute for Air Research.

See all files available:
https://thredds.nilu.no/thredds/catalog/ebas/catalog.html

In [1]:
# You need to first install threddsclient:
# remember to do in terminal pip install threddsclient
import threddsclient
import xarray as xr

In [ ]:
# Find url addresses for files on EBAS

all_opendap_urls = threddsclient.opendap_urls(
'https://thredds.nilu.no/thredds/catalog/ebas/catalog.xml')

In [ ]:
#Example 1 nephelometer scattering coefficient data
# get all data urls for one station, e.g., Zeppelin NO0042G
opendap_urls = [x for x in all_opendap_urls if 'NO0042G' in x]
# get all scattering data urls
opendap_urls = [x for x in opendap_urls if 'nephelometer' in x]

opendap_urls

In [ ]:
# read multiple files
# the problem with this example is, 
#    that the files are from different instruments
#    thus, they need to be treated and digested one by one.

dsmf = xr.open_mfdataset(opendap_urls)
dsmf

In [ ]:
# Example 2 aerosol sulphate data 
# also here - 
#    the files are differently shaped for different periods
#    best to read single files, analyse, and then concatenate for longer time series
#    note that catalogue file names give no full description of what is in a file

# get all data urls for one station, e.g., Zeppelin NO0042G
opendap_urls = [x for x in all_opendap_urls if 'NO0042G' in x]

# get all data urls which could contain sulphate data, exclude some
opendap_urls = [x for x in opendap_urls if 'filter_3pack' in x]
opendap_urls = [x for x in opendap_urls if not 'sum_ammonia' in x]
opendap_urls = [x for x in opendap_urls if not 'sulphur_dioxide' in x]

opendap_urls

In [ ]:
file_to_read = opendap_urls[0]
file_to_read

In [ ]:
ds_single_file = xr.open_dataset(file_to_read)
ds_single_file

In [ ]:
ds_single_file['sulphate_total_ug_S_per_m3'].plot()